<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:37px'>Primeiros Passos</h1>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
<ul style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    <li>Abrir o terminal do linux</li>
    <li>Digitar: <strong> source activate  </strong></li>
    <li>Digitar: <strong> pip install textblob  </strong></li>
    <li>Digitar: <strong> pip install wordcloud  </strong></li>
    <li>Digitar: <strong> git clone https://github.com/hermino/mini_curso_nlp.git </strong></li>
    <li>Digitar: <strong> cd /mini_curso_nlp </strong></li>
    <li>Digitar: <strong> jupyter notebook  </strong></li>
</ul>
</div>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    <strong> O que é jupyter notebook? </strong>
    </p>
<div>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:37px'>Introdução a Processamento de Linguagem Natural</h1>

<img src="imgs/head.png"/>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:20px'>O que é Processamento de Linguagem Natual?</h1>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    O Processamento de Linguagem Natural (PLN) é a subárea da Inteligência Artificial (IA) que estuda a capacidade e as limitações de uma máquina em entender a linguagem dos seres humanos. 
    </p>
<div>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:20px'>Quais seus objetivos?</h1>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    O objetivo do PLN é fornecer aos computadores a capacidade de entender e compor textos. 
    </p>
<div>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:20px'>Pode ser usada para?</h1>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
<ul style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    <li>Reconhecer o contexto</li>
    <li>Fazer análises sintáticas, semânticas, léxicas e morfológicas</li>
    <li>Criar resumos</li>
    <li>Extrair informação</li>
    <li>Interpretar os sentidos</li>
    <li><strong>Analisar sentimentos</strong></li>
    <li>Aprender conceitos</li>
</ul>
</div>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:37px'>Análise de sentimentos em textos</h1> 

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <ol style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
        <li>Este minicurso terá como base os comentário da Amazon, Yelp (estabelecimentos comerciais) e IMDb.</li>
        <li>Cada dataset contém registros, sendo 1 para reviews Positivos e 0 para Negativos.</li>
    </ol>
<div>

In [ ]:
# Para analise de textos
import nltk
from textblob import TextBlob

# Manipulação de dados
import numpy as np
import pandas as pd

# Criação de Gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# Apredizado de Máquina e PLN
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Detalhes de visualização
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Explorando os dados</h1> 

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    O primeiro passo antes qualquer análise é primeiramente verificar quais as caracteristicas dos seus dados. O intuito é encontrar dados que podem ser importantes para a análise. 
    </p>
<div>

In [ ]:
arquivos_dict = {'amazon': './bases/amazon_cells_labelled.txt',
                 'yelp': './bases/yelp_labelled.txt',
                 'imdb': './bases/imdb_labelled.txt'}

df_list = []

for fonte, arquivos in arquivos_dict.items():
    df = pd.read_csv(arquivos, names=['sentence', 'label'], sep='\t')
    df['fonte'] = fonte
    df_list.append(df)
    
df = pd.concat(df_list)
print('Formato do arquivo {}'.format(df.shape))

In [ ]:
df.sample(10)

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Verificando a quantidades de amostras com valores nulos</h1> 

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)
df.sample(10)

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    Mesmo verificando os dados nulos com a função <strong>isnull()</strong>, em alguns casos os datasets de textos apresentam campos em branco. 
    </p>
<div>


In [ ]:
brancos = []

for indice, senteca, label, fonte in df.itertuples():
    if(senteca.isspace()): # retorna um TRUE se houver apenas um caractere em branco
        brancos.append(i)
        
brancos

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Vamos plotar um WordCloud com os reviews</h1> 

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stopwordsNltk = stopwords.words('english')

from wordcloud import WordCloud

words = []

for senteca in df.sentence:
    for palavra in senteca.lower().split():
        if(palavra not in stopwordsNltk):
            words.append(palavra)
            
words = str(words)

wordcloud = WordCloud(width=1000, height=1000, margin=0).generate(words)
plt.figure(figsize=(20,11))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.margins(x=0, y=0)

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Antes de iniciarmos uma análise mais profunda e técnica, podemos procurar padrões “suspeitos” em nosso dataset</h1>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
<ul style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    <li>Será que existe diferença de tamanho (caracteres) do comentário entre os datasets?</li>
    <li>Podemos verificar alguma correlação tamanho do review x sentimento?</li>
    <li>A forma de escrita do comentário (exemplos: educado, rude, gírias, etc…) tem influência no sentimento <strong>positivo</strong> ou <strong>negativo</strong>?</li>
</ul>
</div>
<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
   Vamos descobrir através da vizualização dos dados 
    </p>
<div>


In [ ]:
df['tamanho_texto'] = df['sentence'].apply(len)
df.head()

In [ ]:
x1 = list(df[df['fonte'] == 'amazon']['tamanho_texto'])
x2 = list(df[df['fonte'] == 'yelp']['tamanho_texto'])
x3 = list(df[df['fonte'] == 'imdb']['tamanho_texto'])

colors = ['#fb8072', '#80b1d3', '#fdb462']
names = ['Amazon', 'Yelp', 'Imdb']

plt.figure(figsize=(9,7))
plt.hist([x1, x2, x3], bins = int(180/15), normed=False, color = colors, label=names)

plt.xlabel('Quantidade de caracteres')
plt.ylabel('Quantidade de comentários')
plt.title('Histograma dos Reviews')
plt.legend();

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    Verificando rapidamente o histograma acima, não parece haver distinção entre os Datasets, apenas o Imdb apresenta alguns comentários mais longos que os demais. Mas será que isso influencia no sentimento do comentário: Positivo ou Negativo?
    </p>
</div>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    Vamos verificar...
    </p>
</div>

In [ ]:
plt.figure(figsize=(11, 9))
sns.swarmplot(x='label', y='tamanho_texto', data=df, hue='fonte', alpha=0.7, palette='coolwarm')
plt.xlabel('Sentimento do comentário');

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    Conforme mostra nosso swarmplot, não há relação entre o tamanho do comentário com a probabilidade de ser Positivo ou Negativo.
    </p>
</div>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Vamos começar efetivamente a análise de sentimento com dataset da Amazon!</h1>

<img src="./imgs/amazon.png"/>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
<ul style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    <li>A base de dados será dividida em 80/20, sendo respectivamente: Treino e Teste</li>
    <li>Utilizado Stemming — Refere-se a um processo heurístico não muito refinado que corta as extremidades das palavras na esperança de atingir esse objetivo corretamente na maior parte do tempo, e geralmente inclui a remoção de afixos derivados</li>
</ul>
</div>

In [ ]:
amazon = df[df['fonte'] == 'amazon']
amazon = amazon[['sentence', 'label']]
print('Formato do Dataframe da Amazon {}'.format(amazon.shape))
amazon.head()

In [ ]:
amazon['sentence'] = amazon['sentence'].str.replace('[^\w\s]','')
amazon.head()

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Divisão da base de dados</h1>

In [ ]:
from sklearn.model_selection import train_test_split
treino, teste = train_test_split(amazon, test_size=0.2)

In [ ]:
print('Shape dos dados de Treino {}'.format(treino.shape))
treino.label.value_counts(normalize=True)

In [ ]:
print('Shape dos dados de Teste {}'.format(teste.shape))
teste.label.value_counts(normalize=True)

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Aplicando Stemmer</h1>

In [ ]:
treino = [tuple(x) for x in treino.values]

In [ ]:
def aplica_stemmer(texto):
    stemmer = nltk.stem.SnowballStemmer('english')
    
    frases_com_stemming = []
    for (palavras, sentimento) in texto:
        com_stemming = [str(stemmer.stem(p)) for p in palavras.lower().split() if p not in stopwordsNltk]
        frases_com_stemming.append((com_stemming, sentimento))
    return frases_com_stemming

In [ ]:
frases_com_stem_treinamento = aplica_stemmer(treino)

In [ ]:
pd.DataFrame(frases_com_stem_treinamento, columns=['frase', 'sentimento']).sample(10)

In [ ]:
teste = [tuple(x) for x in teste.values]
frases_com_stem_teste = aplica_stemmer(teste)

In [ ]:
def busca_palavras(frases):
    todas_palavras = []
    for (palavras, sentimento) in frases:
        todas_palavras.extend(palavras)
    return todas_palavras

In [ ]:
palavras_treinamento = busca_palavras(frases_com_stem_treinamento)
palavras_teste = busca_palavras(frases_com_stem_teste)

In [ ]:
print('Quantidade de palavras na base de treinamento {}'.format(pd.DataFrame(palavras_treinamento).count()))

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Listando palavras mais populares</h1>

In [ ]:
def busca_frequencia(palavras):
    palavras = nltk.FreqDist(palavras)
    return palavras

In [ ]:
frenquencia_treinamento = busca_frequencia(palavras_treinamento)
frenquencia_treinamento.most_common(10)

In [ ]:
frenquencia_teste = busca_frequencia(palavras_teste)
frenquencia_teste.most_common(10)

In [ ]:
plt.figure(figsize=(12,8))
frenquencia_treinamento.plot(30, cumulative=False);

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Buscando palavras unicas</h1>

In [ ]:
def busca_palavras_unicas(frequencia):
    freq = frequencia.keys()
    return freq

palavras_unicas_treinamento = busca_palavras_unicas(frenquencia_treinamento)
palavras_unicas_teste = busca_palavras_unicas(frenquencia_teste)

In [ ]:
def extrator_palavras(documento):
    doc = set(documento) #coleção não ordenada e não indexada
    caracteristicas  = {}
    for palavras in palavras_unicas_treinamento:
        caracteristicas['%s' % palavras] = (palavras in doc)
    return caracteristicas

In [ ]:
def extrator_palavras_teste(documento):
    doc = set(documento) #coleção não ordenada e não indexada
    caracteristicas  = {}
    for palavras in palavras_unicas_teste:
        caracteristicas['%s' % palavras] = (palavras in doc)
    return caracteristicas

In [ ]:
base_completa_treinamento = nltk.classify.apply_features(extrator_palavras, frases_com_stem_treinamento)
base_completa_teste = nltk.classify.apply_features(extrator_palavras_teste, frases_com_stem_teste)

In [ ]:
base_completa_treinamento

In [ ]:
classificador = nltk.NaiveBayesClassifier.train(base_completa_treinamento)
print(classificador.labels())

In [ ]:
print('Acurácia: {}%'.format(nltk.classify.accuracy(classificador, base_completa_teste)))

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Como melhorar o modelo?</h1>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
<ul style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    <li>Aumentar a base de dados para o treino e teste, com isso teríamos um modelo mais robusto</li>
    <li>Testar outros algoritmos e hiperparâmetros</li>
    <li>Incluir novas StopWords…</li>
</ul>
</div>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Verificando os erros do modelo</h1>

In [ ]:
erros = []
for (frase, classe) in base_completa_teste:
    resultado = classificador.classify(frase)
    if resultado != classe:
        erros.append((classe, resultado, frase))

In [ ]:
esperado = []
previsto = []
for (frase, classe) in base_completa_teste:
    resultado = classificador.classify(frase)
    previsto.append(resultado)
    esperado.append(classe)

    
font = {'family': 'DejaVu Sans',
        'weight': 'bold',
        'size': 16}    

plt.rc('font', **font)       
cm = confusion_matrix(esperado, previsto)
plt.figure(figsize=(12, 8))
plt.clf()
plt.imshow(cm, interpolation='nearest', cmap='Blues')
className = ['Negativa','Positiva']
plt.title('Matriz de Confusão Amazon')
plt.ylabel('Classe Verdadeira')
plt.xlabel('Classe Prevista')
tick_marks = np.arange(len(className))
plt.xticks(tick_marks, className, rotation=45)
plt.yticks(tick_marks, className);
s = [['TN','FP'], ['FN', 'TP']]

for i in range(2):
    for j in range(2):
        plt.text(j, i, str(s[i][j] + " = " + str(cm[i][j])))
plt.show();

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Usando o NLTK para análisar o Sentimento</h1>

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for line in amazon.sentence:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)

print(results[:3])

In [ ]:
sentimentos = pd.DataFrame.from_records(results)
sentimentos.head()

In [ ]:
sentimentos['label'] = 0
sentimentos.loc[sentimentos['compound'] > 0.2, 'label'] = 1
sentimentos.loc[sentimentos['compound'] < -0.2, 'label'] = -1
sentimentos.head()

In [ ]:
amazon.index = range(len(amazon))

In [ ]:
accuracy_score(amazon.label, sentimentos.label)

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Probabilidade de cada classe</h1>

In [ ]:
teste = 'i love the movie'
testeStemming = []
stemmer = nltk.stem.SnowballStemmer('english')

for (palavras_treinamento) in teste.split():
    comStem = [p for p in palavras_treinamento.split()]
    testeStemming.append(str(stemmer.stem(comStem[0])))
    
novo = extrator_palavras(testeStemming)
distribuicao = classificador.prob_classify(novo)

for classe in distribuicao.samples():
    print('%s: %f' % (classe, distribuicao.prob(classe)))
    if classe == 0:
        print('Probalidade de ser Negativa')
    else:
        print('Probalidade de ser Positiva')
        

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Agora vamos realizar a análise de sentimentos da Yelp</h1>

<img style="width: 500px]; heigth: 500px" src="./imgs/yelp.png"/>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Utilizando TF-IDF</h1>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
No caso do TFIDF, o único pré-processamento de texto é remover a pontuação e diminuir as letras maiúsculas e minúsculas. Nós não temos que nos preocupar com as StopWords. O TFIDF é a junção do TF e IDF.
    </p>
</div>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
<ul style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
    <li>O TFIDF é usado com sucesso pelos mecanismos de pesquisa, como o Google, como um fator de classificação para o conteúdo.</li>
    <li>A ideia é pesar os termos freqüentes e aumentar os mais raros</li>
</ul>
</div>

<div style='padding:20px; border: 1px solid #FFDAB9; border-radius: 5px; background: #FFDAB9'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
Nossa acurácia para este modelo foi de 77,5% utilizando SGDClassifier.
    </p>
</div>    

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Matriz de Confusão</h1>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Agora vamos analisar o Imdb</h1>

<div style='padding:20px; border: 1px solid #FFFACD; border-radius: 5px; background: #FFFACD'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
        <strong>Obs:</strong> Nós poderíamos realizar uma única análise de sentimentos com o dataset consolidade que foi importado (concatenação) de todos os conjuntos, porém, o notebook ficará mais completo e didático se tratarmos de forma separada...<br><br> 
        Mãos a obra!
    </p>
</div> 

<img style="width=500px; height: 500px" src="./imgs/imdb.png"/>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Separando os Dados</h1>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Removendo StopWords</h1>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Análise Morfológica
</h1>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
Site que explica cada sigla: https://www.clips.uantwerpen.be/pages/pattern-en
    </p>
</div>

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Classificação e Ajuste de Hiperparâmetros
</h1>

<div style='padding:20px; border: 1px solid #FFFACD; border-radius: 5px; background: #FFFACD'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
        Como o melhor C encontrado pelo GridSearch foi o C = 1, não precisamos deixar explícito pois o default é este valor.
    </p>
</div> 

<h1 style='font-family:ubuntu; color:#1C1C1C; font-size:25px'>Terminamos nossa análise de sentimentos para os 3 Datasets</h1>

<div style='padding:20px; border: 1px solid #E0FFFF; border-radius: 5px; background: #E0FFFF'>
    <p style='font-family:Ubuntu Mono; color:#1C1C1C; font-size:18px'>
Para fechar, vamos realizar a análise de sentimentos com um único Dataset, 
    sem diferenciar a fonte dos dados.
    </p>
</div>